In [18]:
#0: Import and install dependencies
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ttest_1samp, pearsonr
from scipy import odr

#0.1: Create all functions
def diff_A1C_set(training=None):
    flag = 0
    for ID in training.ID.unique():
        training_ = training.loc[training.ID == ID]
        if len(training_)>1:
            diffsMeans = training_.Mean.values[1:]-training_.Mean.values[:-1]
            diffsA1Cs = training_.A1C.values[1:]-training_.A1C.values[:-1]
            hba1cs_now = training_.A1C[1:]
            hba1cs_past = training_.A1C[1:]
            means = training_.Mean[1:]
            devices = training_.Device[1:]
            if flag == 0:
                flag = 1
                final = pd.DataFrame([diffsMeans,
                                      diffsA1Cs,
                                      hba1cs_now,
                                      hba1cs_past,
                                      means,
                                      devices]).T
            else:
                final_ = pd.DataFrame([diffsMeans,
                                      diffsA1Cs,
                                      hba1cs_now,
                                      hba1cs_past,
                                      means,
                                      devices]).T
                final = pd.concat([final,final_])
    final.columns = 'diffMeans','diffA1Cs','A1Cnow','A1Cpast','Mean','Device'
    return final

def summary_devices(device,x_val,y_val):
    summ =  np.asarray([r2(x_val,y_val),
                                  bias_BA(x_val,y_val),bias_p(x_val,y_val),
                                  std_BA(x_val,y_val)[0],std_BA(x_val,y_val)[1],
                                  r_bias(x_val,y_val),r_bias_p(x_val,y_val),rms(x_val,y_val)])
    return summ

def r2(x_val,y_val):
    correlation_matrix = np.corrcoef(x_val, y_val)
    correlation_xy = correlation_matrix[0,1]
    r2 = correlation_xy**2
    return np.round(r2,2)

def rms(x_val,y_val):
    return np.round(np.sqrt(np.mean((y_val-x_val)**2)),4)

def bias_BA(x_val,y_val):
    diffs = y_val - x_val
    return np.round(np.mean(diffs),4)

def bias_p(x_val,y_val):
    return np.round(ttest_1samp(y_val-x_val,0)[1],4)

def std_BA(x_val,y_val):
    means = np.mean([y_val, x_val], axis=0)
    diffs = y_val - x_val
    mean_diff = np.mean(diffs)
    limit_of_agreement = 1.96*np.std(diffs, axis=0)
    return np.round(mean_diff - limit_of_agreement,4), np.round(mean_diff + limit_of_agreement,4)

def r_bias(x_val,y_val):
    correlation_matrix = np.corrcoef(x_val,y_val-x_val)
    correlation_xy = correlation_matrix[0,1]
    return np.round(correlation_xy,2)

def r_bias_p(x_val,y_val):
    return np.round(pearsonr(x_val,y_val-x_val)[1],4)

def f(B,x):
    return B[0]*x

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the data file
downloaded = drive.CreateFile({'id':"1Fg6KptqUa3FWb0S5A8AX2KQuLNkL175Z"})   # replace the id with id of file you want to access
downloaded.GetContentFile('data.csv')

#3: Read data to Pandas
data = pd.read_csv('data.csv')

#4: Divide data to training, testing and validation datasets
training = diff_A1C_set(data.loc[data.Source==0].copy())
testing = diff_A1C_set(data.loc[data.Source==1].copy())
validation = diff_A1C_set(data.loc[data.Source==2].copy())
external = diff_A1C_set(data.loc[data.Source==3].copy())

#5: Estimate the Beta for Deming's weighted regression where ratio of error variances = 1 (implementation as orthogonal distance regression in SciPy)
linear = odr.Model(f)
odrdata = odr.Data(training['diffMeans'],training['diffA1Cs'])
myodr = odr.ODR(odrdata, linear, beta0=[1])
myoutput = myodr.run()
print('Beta = {}'.format(1/myoutput.beta[0]))

#6: Create the summary of Deming's regression on training, testing and validation datasets
flag = 0
for data_ in [training,testing,validation,external]:
    data_['Deming'] = data_.A1Cpast.values+data_.diffMeans.values*myoutput.beta[0]
    for dtype in np.concatenate([[3],data_.Device.unique()]):
        data__ = data_.loc[data_.Device==dtype].copy()
        if dtype==3:
            data__ = data_
        summary = pd.DataFrame(summary_devices(data__.Device,data__.Deming,data__.A1Cnow)).T
        summary.columns = 'R2','Bias','p for bias difference from 0','95%CI(low)','95%CI(high)','r (bias, HbA1c)','p (bias, HbA1c)','RMSE'
        if flag==0:
            flag = 1
            summaryfull = summary
        else:
            summaryfull = pd.concat([summaryfull,summary],axis=0)

summaryfull = summaryfull[:-1]
summaryfull.index = 'Training All','Training Medtronic','Training Abbott','Testing All','Testing Medtronic','Testing Abbott','Validation All','Validation Medtronic','Validation Abbott','External All'
summaryfull

Beta = 44.853082844000994


,R2,Bias,p for bias difference from 0,95%CI(low),95%CI(high),"r (bias, HbA1c)","p (bias, HbA1c)",RMSE
Training All,0.91,-0.0328,0.0116,-0.5231,0.4575,-0.49,0.0000,0.2523
Training Medtronic,0.88,-0.0456,0.0109,-0.6033,0.5121,-0.53,0.0000,0.2882
Training Abbott,0.96,-0.0050,0.7134,-0.2920,0.2820,-0.33,0.0002,0.1465
Testing All,0.92,-0.0145,0.2807,-0.4364,0.4073,-0.46,0.0000,0.2157
Testing Medtronic,0.90,-0.0178,0.3286,-0.4876,0.4520,-0.52,0.0000,0.2404
Testing Abbott,0.96,-0.0076,0.6502,-0.3016,0.2865,-0.30,0.0064,0.1502
Validation All,0.92,0.0137,0.6476,-0.4236,0.4510,-0.30,0.0224,0.2235
Validation Medtronic,0.90,0.0508,0.2326,-0.4338,0.5353,-0.25,0.1350,0.2524
Validation Abbott,0.96,-0.0498,0.1652,-0.3531,0.2534,-0.43,0.0519,0.1625
External All,0.88,-0.0100,0.2769,-0.7529,0.7329,-0.50,0.0000,0.3792
